In [25]:
from sqlalchemy import Column
from sqlalchemy import ForeignKey
from sqlalchemy import Integer
from sqlalchemy import String
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import relationship

In [26]:
Base = declarative_base()

class Mutante(Base):
    __tablename__ = "mutante"
    idMutante = Column(Integer, primary_key=True)
    nombre = Column(String(70))
    tipo = Column(String(10))
    poderes = relationship("Poder", back_populates="mutante",
                           cascade="all,delete-orphan")
    
    def __repr__(self):
        return f"Mutante: id={self.idMutante!r}, nombre={self.nombre!r}), tipo={self.tipo!r}"
    
class Poder(Base):
    __tablename__ = "Poder"
    idPoder = Column(Integer, primary_key=True)
    descripcion = Column(String(200), nullable=False)
    idmutante = Column(Integer, ForeignKey("mutante.idMutante"), nullable = False)
    mutante = relationship("Mutante", back_populates="poderes")
    
    def __repr__(self):
        return f"Poder(id={self.idPoder!r}, descripcion={self.descripcion!r})"

In [27]:
from sqlalchemy import create_engine

engine = create_engine("mysql+pymysql://root:@localhost/mutantes123?charset=utf8mb4", echo=True)

In [28]:
# Crear la base de datos.
Base.metadata.create_all(engine)

2022-10-21 08:27:09,822 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2022-10-21 08:27:09,822 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-10-21 08:27:09,825 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2022-10-21 08:27:09,826 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-10-21 08:27:09,828 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2022-10-21 08:27:09,830 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-10-21 08:27:09,833 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-21 08:27:09,834 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-10-21 08:27:09,835 INFO sqlalchemy.engine.Engine [generated in 0.00092s] {'table_schema': 'mutantes123', 'table_name': 'mutante'}
2022-10-21 08:27:09,882 INFO sqlalchemy.engine.Engine SELECT COUNT(*) FROM information_schema.tables WHERE table_schema = %(table_schema)s AND table_name = %(table_name)s
2022-10-21 08:27:09,8

In [38]:
#insertar registros

from sqlalchemy.orm import Session

with Session(engine) as session:
    #creamos nuestros mutantes 
    iceman = Mutante(nombre="Iceman", tipo=" BUENO", poderes = [Poder(descripcion="Congelar objetos")])
    astroboy = Mutante(nombre="Astroboy", tipo=" BUENO", poderes = [Poder(descripcion="Volar"), Poder(descripcion="Lazer")])

    #agregamos los mutantes a la sesion
    session.add_all([iceman, astroboy])

    #commit
    session.commit()

2022-10-21 08:43:31,843 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-21 08:43:31,845 INFO sqlalchemy.engine.Engine INSERT INTO mutante (nombre, tipo) VALUES (%(nombre)s, %(tipo)s)
2022-10-21 08:43:31,847 INFO sqlalchemy.engine.Engine [cached since 981.9s ago] {'nombre': 'Iceman', 'tipo': ' BUENO'}
2022-10-21 08:43:31,849 INFO sqlalchemy.engine.Engine INSERT INTO mutante (nombre, tipo) VALUES (%(nombre)s, %(tipo)s)
2022-10-21 08:43:31,850 INFO sqlalchemy.engine.Engine [cached since 981.9s ago] {'nombre': 'Astroboy', 'tipo': ' BUENO'}
2022-10-21 08:43:31,852 INFO sqlalchemy.engine.Engine INSERT INTO `Poder` (descripcion, idmutante) VALUES (%(descripcion)s, %(idmutante)s)
2022-10-21 08:43:31,853 INFO sqlalchemy.engine.Engine [cached since 981.9s ago] {'descripcion': 'Congelar objetos', 'idmutante': 7}
2022-10-21 08:43:31,855 INFO sqlalchemy.engine.Engine INSERT INTO `Poder` (descripcion, idmutante) VALUES (%(descripcion)s, %(idmutante)s)
2022-10-21 08:43:31,855 INFO sqlalchemy.e

In [30]:
#Seleccionar datos
from sqlalchemy import select

session = Session(engine)

stmt = select(Mutante).where(Mutante.nombre.in_(["Iceman", "Astroboy"]))

for user in session.execute(stmt):
    print(user)

2022-10-21 08:27:10,001 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-21 08:27:10,005 INFO sqlalchemy.engine.Engine SELECT mutante.`idMutante`, mutante.nombre, mutante.tipo 
FROM mutante 
WHERE mutante.nombre IN (%(nombre_1_1)s, %(nombre_1_2)s)
2022-10-21 08:27:10,006 INFO sqlalchemy.engine.Engine [generated in 0.00152s] {'nombre_1_1': 'Iceman', 'nombre_1_2': 'Astroboy'}
(Mutante: id=1, nombre='Iceman'), tipo=' BUENO',)
(Mutante: id=2, nombre='Astroboy'), tipo=' BUENO',)
(Mutante: id=3, nombre='Iceman'), tipo=' BUENO',)
(Mutante: id=4, nombre='Astroboy'), tipo=' BUENO',)
(Mutante: id=5, nombre='Iceman'), tipo=' BUENO',)
(Mutante: id=6, nombre='Astroboy'), tipo=' BUENO',)


In [31]:
#Seleccionar datos de dos tablas

stmt = (select(Poder).join(Poder.mutante).where(Mutante.nombre == "Astroboy"))

astroboy_powers = session.execute(stmt).scalars().all()

astroboy_powers



2022-10-21 08:27:10,052 INFO sqlalchemy.engine.Engine SELECT `Poder`.`idPoder`, `Poder`.descripcion, `Poder`.idmutante 
FROM `Poder` INNER JOIN mutante ON mutante.`idMutante` = `Poder`.idmutante 
WHERE mutante.nombre = %(nombre_1)s
2022-10-21 08:27:10,054 INFO sqlalchemy.engine.Engine [generated in 0.00185s] {'nombre_1': 'Astroboy'}


[Poder(id=2, descripcion='Volar'),
 Poder(id=3, descripcion='Lazer'),
 Poder(id=5, descripcion='Volar'),
 Poder(id=6, descripcion='Lazer'),
 Poder(id=8, descripcion='Volar'),
 Poder(id=9, descripcion='Lazer')]

In [32]:
#modificar datos

astroboy_powers[1].descripcion = "Laser"
session.commit()


2022-10-21 08:27:10,206 INFO sqlalchemy.engine.Engine UPDATE `Poder` SET descripcion=%(descripcion)s WHERE `Poder`.`idPoder` = %(Poder_idPoder)s
2022-10-21 08:27:10,210 INFO sqlalchemy.engine.Engine [generated in 0.00385s] {'descripcion': 'Laser', 'Poder_idPoder': 3}
2022-10-21 08:27:10,253 INFO sqlalchemy.engine.Engine COMMIT


In [37]:
#Borrar datos

session.delete(astroboy_powers[0])  
session.commit()

2022-10-21 08:28:30,746 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-21 08:28:30,749 INFO sqlalchemy.engine.Engine DELETE FROM `Poder` WHERE `Poder`.`idPoder` = %(idPoder)s
2022-10-21 08:28:30,749 INFO sqlalchemy.engine.Engine [cached since 28.98s ago] {'idPoder': 2}
2022-10-21 08:28:30,751 INFO sqlalchemy.engine.Engine COMMIT


/var/folders/_x/5t3wtggd6n991cckzsst5wh40000gn/T/ipykernel_91666/1209737355.py:4: SAWarning: DELETE statement on table 'Poder' expected to delete 1 row(s); 0 were matched.  Please set confirm_deleted_rows=False within the mapper configuration to prevent this warning.
  session.commit()
